# Goal:
Use Divvy Dataset to build an MLOps process for the MLOps course


# Step-by-step approach

## Load the data

## Explore and clean

## Define a target for ML training

## Define a ML model

## Define features to use



# download the data



In [1]:
import requests
import datetime
import pandas as pd
import matplotlib as plt
from tqdm import tqdm
import zipfile
import os

In [2]:
files= [('202301','./data'),('202302','./data')]

for file, path in files:
    url=f'https://divvy-tripdata.s3.amazonaws.com/{file}-divvy-tripdata.zip'
    resp=requests.get(url, stream=True)
    zip_save_path = f'{path}/{file}.zip'

    os.makedirs(path, exist_ok=True)

    with open(zip_save_path,"wb") as handle:
        for data in tqdm(resp.iter_content(chunk_size=1024),
                         desc=f'{file}',
                         postfix=f"save to {zip_save_path}",
                         total=int(resp.headers["Content-Length"])):
            handle.write(data)

    with zipfile.ZipFile(zip_save_path, 'r') as zip_ref:
        zip_ref.extractall(path)
    
    os.remove(zip_save_path)

202302:   0%|          | 6916/7081095 [00:00<09:29, 12429.35it/s, save to ./data/202302.zip]


In [3]:
data_jan = pd.read_csv("data/202301-divvy-tripdata.csv")

In [4]:
data_jan.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,F96D5A74A3E41399,electric_bike,2023-01-21 20:05:42,2023-01-21 20:16:33,Lincoln Ave & Fullerton Ave,TA1309000058,Hampden Ct & Diversey Ave,202480.0,41.924074,-87.646278,41.930000,-87.640000,member
1,13CB7EB698CEDB88,classic_bike,2023-01-10 15:37:36,2023-01-10 15:46:05,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
2,BD88A2E670661CE5,electric_bike,2023-01-02 07:51:57,2023-01-02 08:05:11,Western Ave & Lunt Ave,RP-005,Valli Produce - Evanston Plaza,599,42.008571,-87.690483,42.039742,-87.699413,casual
3,C90792D034FED968,classic_bike,2023-01-22 10:52:58,2023-01-22 11:01:44,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member
4,3397017529188E8A,classic_bike,2023-01-12 13:58:01,2023-01-12 14:13:20,Kimbark Ave & 53rd St,TA1309000037,Greenwood Ave & 47th St,TA1308000002,41.799568,-87.594747,41.809835,-87.599383,member


In [7]:
data_jan.describe()

,start_lat,start_lng,end_lat,end_lng
count,190301.000000,190301.000000,190174.000000,190174.000000
mean,41.897081,-87.647161,41.897158,-87.647265
std,0.049169,0.028476,0.049226,0.028516
min,41.648501,-87.830000,41.648501,-87.840000
25%,41.878223,-87.661000,41.878317,-87.661206
50%,41.894618,-87.643969,41.894556,-87.644098
75%,41.926277,-87.630000,41.926277,-87.630000
max,42.070000,-87.528232,42.080000,-87.528232


In [5]:
# missing data for started at or station name
data_jan.isnull().sum()

ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_station_name    26721
start_station_id      26721
end_station_name      27840
end_station_id        27840
start_lat                 0
start_lng                 0
end_lat                 127
end_lng                 127
member_casual             0
dtype: int64

In [8]:
#removing rows without station name
data_jan=data_jan.dropna(subset=['start_station_name'])




In [9]:
data_jan.describe()

,start_lat,start_lng,end_lat,end_lng
count,163580.000000,163580.000000,163453.000000,163453.000000
mean,41.896495,-87.645415,41.896638,-87.645761
std,0.046748,0.026131,0.046890,0.026422
min,41.648501,-87.830000,41.648501,-87.840000
25%,41.878166,-87.658818,41.878317,-87.659657
50%,41.893992,-87.642817,41.893992,-87.642884
75%,41.924189,-87.629544,41.924816,-87.629634
max,42.064854,-87.528232,42.070000,-87.528232


# Predict demand at start station

## Define target variable
Target will be the number of rental at start station by hour


## Features
1 - Time features extracted from started at (day, day of the week, weekday, weekend)
2 - Ridable type 
3 - start station name



